# Data Wrangling Project

## Objectice: Loading and importing data into Mysql database to  perform queries .


### Map Area: 
**Ahmedabad,India**




### Imports and Data Audit

In [1]:
import os
import collections
import pprint
import xml.etree.cElementTree as ET
import re
import codecs
from unittest import TestCase
import csv
import copy
import cerberus
import schema
import sqlite3
from IPython.display import display, HTML
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

datadir = "/home/sushil/Downloads"
datafile = "ahmedabad.osm"
ahmedabad_osm_file = os.path.join(datadir, datafile)
ahmedabad_osm_file

ModuleNotFoundError: No module named 'cerberus'

### Parse through the Ahmedabad,India dataset with ElementTree and count the number of unique element types to get an overall understanding of the data by using count_tags function.

In [2]:
#Parse through to count the number of unique element types to understand overall structure.

def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

ahm_tags = count_tags(ahmedabad_osm_file)
pprint.pprint(ahm_tags)



NameError: name 'ahmedabad_osm_file' is not defined

### Patterns in the Tags

In [91]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

cal_keys = process_map(ahmedabad_osm_file)
pprint.pprint(cal_keys)

{'lower': 96267, 'lower_colon': 1995, 'other': 34, 'problemchars': 7}


### Unique users contribution to the map in Ahmedabad area.

In [92]:
#people invovlved in the map editing.
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
ahm_users = process_map(ahmedabad_osm_file)
print"The total unique contributors:",len((ahm_users))

The total unique contributors: 359


### Prepare data for Database - SQL
**Build CSV file from OSM. Parse, clean and shape data.**

In [141]:
#creating 5 different csv file for particular element type
Nodes_File = "nodes.csv"
Node_Tags_File = "nodes_tags.csv"
Ways_File= "ways.csv"
Way_Nodes_File = "ways_nodes.csv"
Way_Tags_File = "ways_tags.csv"

Lower_colon = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
Problem_characters = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.Schema

#mentioning the header for each csv file
NodeFields = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
Node_Tag_Fields = ['id', 'key', 'value', 'type']
WayFields = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
Way_Tag_Fields = ['id', 'key', 'value', 'type']
Way_Node_Fields = ['id', 'node_id', 'position']


def elements_shape(element, node_attr_fields = NodeFields, way_attr_fields = WayFields,
                  problem_chars = Problem_characters, default_tag_type = 'regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # In both node and way elements this handles the secondary tags
    if element.tag == 'node':
        for node in NodeFields:
            node_attribs[node] = element.attrib[node]
        for child in element:
            tag = {}
            if Problem_characters.search(child.attrib["k"]):
                continue
        
            elif Lower_colon.search(child.attrib["k"]):
                tag_type = child.attrib["k"].split(':',1)[0]
                tag_key = child.attrib["k"].split(':',1)[1]
                tag["key"] = tag_key
                if tag_type:
                    tag["type"] = tag_type
                else:
                    tag["type"] = 'regular'
            
                tag["id"] = element.attrib["id"]
                tag["value"] = child.attrib["v"]
            else:
                tag["value"] = child.attrib["v"]
                tag["key"] = child.attrib["k"]
                tag["type"] = "regular"
                tag["id"] = element.attrib["id"]
            if tag:
                tags.append(tag)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for way in WayFields:
            way_attribs[way] = element.attrib[way]
        for child in element:
            nd = {}
            tag = {}
            if child.tag == 'tag':
                if Problem_characters.search(child.attrib["k"]):
                    continue
                elif Lower_colon.search(child.attrib["k"]):
                    tag_type = child.attrib["k"].split(':',1)[0]
                    tag_key = child.attrib["k"].split(':',1)[1]
                    tag["key"] = tag_key
                    if tag_type:
                        tag["type"] = tag_type
                    else:
                        tag["type"] = 'regular'
                    tag["id"] = element.attrib["id"]
                    tag["value"] = child.attrib["v"]
    
                else:
                    tag["value"] = child.attrib["v"]
                    tag["key"] = child.attrib["k"]
                    tag["type"] = "regular"
                    tag["id"] = element.attrib["id"]
                if tag:
                    tags.append(tag)
                    
            elif child.tag == 'nd':
                nd['id'] = element.attrib["id"]
                nd['node_id'] = child.attrib["ref"]
                nd['position'] = len(way_nodes)
            
                if nd:
                    way_nodes.append(nd)
            else:
                continue
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = iter(ET.iterparse(ahmedabad_osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def process_users(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    with codecs.open(Nodes_File, 'w') as nodes_file, \
        codecs.open(Node_Tags_File, 'w') as nodes_tags_file, \
        codecs.open(Ways_File, 'w') as ways_file, \
        codecs.open(Way_Nodes_File, 'w') as way_nodes_file, \
        codecs.open(Way_Tags_File, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NodeFields)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, Node_Tag_Fields)
        ways_writer = UnicodeDictWriter(ways_file, WayFields)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, Way_Node_Fields)
        way_tags_writer = UnicodeDictWriter(way_tags_file, Way_Tag_Fields)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = elements_shape(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

if __name__ == '__main__':
    process_users(ahmedabad_osm_file, validate = False)
    sqlite_file = 'project_ahm.db'
    con = sqlite3.connect(sqlite_file)
    con.text_factory = str
    cur = con.cursor()

### Dislpaying data in tabular format
** i.Displaying the contents of node data**

In [142]:
df1=pd.read_csv('/root/nodes.csv')
df1.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,245711024,22.987030,72.494131,vkvora,697960,3,12130592,2012-07-06T13:38:12Z
1,245711047,22.990788,72.380985,katpatuka,17497,3,3689207,2010-01-23T06:03:24Z
2,245711197,23.021624,72.579707,angys,3191182,27,44680444,2016-12-26T11:39:01Z
3,245711252,23.027743,72.600285,PlaneMad,1306,7,28690303,2015-02-07T20:32:10Z
4,245711347,23.051768,72.627162,Harry Wood,1611,2,585956,2008-12-24T18:05:08Z


**ii.Displaying the contents of node_tags data**

In [205]:
df2=pd.read_csv('/root/nodes_tags.csv')
df2.head()

,id,key,value,type
0,245711024,source,AND,regular
1,245711197,ele,53,regular
2,245711197,name,Ahmedabad,regular
3,245711197,place,city,regular
4,245711197,capital,5,regular


**iii.Displaying the contents of Ways data **

In [144]:
df3=pd.read_csv('/root/ways.csv')
df3.head()

,id,user,uid,version,changeset,timestamp
0,22826379,Srihari Thalla,2815653,4,49023075,2017-05-27T11:56:44Z
1,22866823,Maarten Deen,9176,12,12932711,2012-08-31T15:58:03Z
2,22867000,ratrun,474183,24,49595531,2017-06-16T15:58:28Z
3,22867003,kailashdhirwani,4112063,21,41572536,2016-08-20T08:56:44Z
4,28842671,Heinz_V,91490,19,21051608,2014-03-11T19:47:18Z


**iv.Displaying the content of Ways_tags data below**

In [199]:
df4=pd.read_csv('/root/ways_tags.csv')
df4.head()

,id,key,value,type
0,22826379,landuse,residential,regular
1,22866823,ref,SH17,regular
2,22866823,oneway,yes,regular
3,22866823,highway,primary,regular
4,22867000,highway,primary,regular


**v.Displaying the content of Ways_nodes data **

In [146]:
df5=pd.read_csv('/root/ways_nodes.csv')
df5.head()

,id,node_id,position
0,22826379,245711443,0
1,22826379,245711438,1
2,22826379,245711446,2
3,22826379,245711465,3
4,22826379,245711482,4


### The Overview Statistics of The Dataset and Database Queries**

In [147]:
print('The ahmedabad.osm file is {} MB'.format(os.path.getsize('ahmedabad.osm')/1.0e6))
print('The project.db file is {} MB'.format(os.path.getsize('project3.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

The ahmedabad.osm file is 113.845494 MB
The project.db file is 0.360448 MB
The nodes.csv file is 45.16309 MB
The nodes_tags.csv file is 45.16309 MB
The ways.csv file is 4.851864 MB
The ways_tags.csv is 2.980204 MB
The ways_nodes.csv is 15.336533 MB


### Create SQL Database and Tables

In [148]:
# creating table Nodes and insertring values from nodes.csv file
cur.execute("CREATE TABLE Nodes(id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    values = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp'])for i in dr]
cur.executemany("INSERT INTO Nodes VALUES (?, ?, ?, ?, ?, ?, ?, ?);", values)
con.commit()

#create table Nodes_tags and inserting values from nodes_tags.csv file 
cur.execute("CREATE TABLE Nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    values = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
cur.executemany("INSERT INTO Nodes_tags VALUES (?, ?, ?, ?);", values)
con.commit()

#Create table Ways and inserting values from ways.csv file
cur.execute("CREATE TABLE Ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    values = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
cur.executemany("INSERT INTO Ways VALUES (?, ?, ?, ?, ?, ?);", values)
con.commit()

#Create table Ways_nodes inserting values from ways_nodes.csv file
cur.execute("CREATE TABLE Ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    values = [(i['id'], i['node_id'], i['position']) for i in dr]
cur.executemany("INSERT INTO Ways_nodes VALUES (?, ?, ?);", values)
con.commit()

#Create table Ways_tags inserting values from ways_nodes.csv file
cur.execute("CREATE TABLE Ways_tags (id, key, value, type);")
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    values = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
cur.executemany("INSERT INTO Ways_tags VALUES (?, ?, ?, ?);", values)
con.commit()


### Number of Nodes.

In [149]:
cur.execute("SELECT COUNT(*) FROM Nodes;")
print(cur.fetchall())

[(546239,)]


### Number of ways.

In [150]:
cur.execute("SELECT COUNT(*) FROM Ways;")
print(cur.fetchall())

[(81280,)]


### Number of unique users


In [151]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) e;")
print(cur.fetchall())



[(356,)]


### Top 10 contributing users.

In [152]:
import pprint
cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())


[('uday01', 177332),
 ('sramesh', 136709),
 ('chaitanya110', 123135),
 ('shashi2', 49502),
 ('shravan91', 22693),
 ('vkvora', 21967),
 ('shiva05', 19669),
 ('bhanu3', 12515),
 ('Oberaffe', 7094),
 ('kailashdhirwani', 5847)]


### Count Tourism Related Categories Descending

In [165]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM Nodes_tags UNION ALL \
             SELECT * FROM Ways_tags) tags \
             WHERE tags.key LIKE '%tourism'\
             GROUP BY tags.value \
             ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[('attraction', 52),
 ('hotel', 32),
 ('hostel', 20),
 ('guest_house', 14),
 ('information', 9),
 ('museum', 8),
 ('motel', 3),
 ('zoo', 2),
 ('artwork', 1),
 ('viewpoint', 1)]


### Number of coffee chains in the Area

In [154]:
cur.execute ("SELECT Nodes_tags.value, COUNT(*) as num FROM Nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM Nodes_tags WHERE value='cafe') i ON Nodes_tags.id=i.id WHERE Nodes_tags.key='name' \
             GROUP BY Nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[('Cafe Coffee Day', 3),
 ('Barrista, Vijay 4 Roads', 1),
 ('Chai Gate!!!', 1),
 ('Chocolate Room', 1),
 ('Cyber Cafe', 1)]


### Number of different items on the same name in Area.

In [155]:
#Finding how many different items are there on the same name
cur.execute("select value,COUNT(*) as cnt \
             from Nodes_tags where key='name' \
             group by value order by cnt DESC limit 30;")
pprint.pprint(cur.fetchall())


[('tree', 30),
 ('Tree', 15),
 ('Orchir white field', 14),
 ('Palm Greens', 6),
 ('Bank of Baroda', 5),
 ('Memnagar', 4),
 ('Cafe Coffee Day', 3),
 ('Chandkheda', 3),
 ('D Mart', 3),
 ('Fuel', 3),
 ('Gita Mandir', 3),
 ('Maninagar', 3),
 ('Mukhwaas', 3),
 ('Subway', 3),
 ('Temple', 3),
 ('Ahmedabad', 2),
 ('Amul', 2),
 ('BJ Medical College ,Haripura -Asarwa', 2),
 ('Barejadi Railway Station', 2),
 ('Bhadaj', 2),
 ('Bhavsar Hostel', 2),
 ('Bird Feeder', 2),
 ('Central Bank of India', 2),
 ('GMDC', 2),
 ('Gayatri Mandir', 2),
 ('Gurukul', 2),
 ('HDFC Bank', 2),
 ('HDFC Bank Relief Road Branch', 2),
 ('Income Tax Office', 2),
 ('Indian Bank', 2)]


### Top 5 Most Popular Fast Food Chain

In [156]:
cur.execute ("SELECT Nodes_tags.value, COUNT(*) as num FROM Nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM Nodes_tags WHERE value='fast_food') i ON Nodes_tags.id=i.id WHERE Nodes_tags.key='name' \
             GROUP BY Nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[('TGB', 2),
 ('Amul', 1),
 ('Asharfilal Kulfi', 1),
 ('Gol Limbda Bhajiya and Nashta House', 1),
 ('Jasuben Pizza', 1)]


### No of Oneways in the Area

In [202]:
cur.execute("select value,COUNT(*) as cnt \
             from Ways_tags where key='oneway';")
pprint.pprint(cur.fetchall())


[('no', 1752)]


In [204]:
import folium




## Conclusion:

From the above analysis exploring Ahmedabad city is much easier and we will be able to find out several informations about the city,The city Ahmedabad has 546239 Nodes and 81280 Ways in which 1752 are oneways, we an also find out the number of Fastfood chains in the city as well as several other information about the city, but we cannot predict this as accurate analysis because of human contribution and the result may vary.